In [8]:
%run 01_Model_CFE.ipynb


c:\Users\Skole\AppData\Local\Programs\Python\Python39\lib\site-packages\nbformat\__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


ADS: frdp_c --> AMOR + ppi_c
CPR1_CYP71AV1: AMOR + h_c + nadph_c + o2_c --> AAOH + h2o_c + nadp_c
ADH1: AAOH + nad_c --> AAld + h_c + nadh_c
ALDH1: AAld + h2o_c + nadp_c --> AA + h_c + nadph_c


In [9]:
import numpy as np
from tqdm import tqdm

from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
import cobra
from cobra.io import load_model
iMM904 = load_model('iMM904')

In [36]:
def add_dynamic_bounds(iMM904, y):
    """Use external concentrations to bound the uptake flux of sucrose."""
    biomass, GRTT = y  # expand the boundary species
    GRTT_max_import = -10 * GRTT / (5 + GRTT)
    iMM904.reactions.GRTT.lower_bound = GRTT_max_import


def dynamic_system(t, y):
    """Calculate the time derivative of external species."""

    biomass, oxygen = y  # expand the boundary species

    # Calculate the specific exchanges fluxes at the given external concentrations.
    with iMM904:
        add_dynamic_bounds(iMM904, y)

        cobra.util.add_lp_feasibility(iMM904)
        feasibility = cobra.util.fix_objective_as_constraint(iMM904)
        lex_constraints = cobra.util.add_lexicographic_constraints(
            iMM904, ['BIOMASS_SC5_notrace', 'GRTT'], ['max', 'max'])
    fluxes = lex_constraints.values
    fluxes *= biomass
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes


In [37]:
dynamic_system.pbar = None

In [38]:

def infeasible_event(t, y):
    with iMM904:

        add_dynamic_bounds(iMM904, y)

        cobra.util.add_lp_feasibility(iMM904)
        feasibility = cobra.util.fix_objective_as_constraint(iMM904)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [39]:
ts = np.linspace(0, 15, 100)  # Desired integration resolution and interval
y0 = [0.1, 10]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )

    

t = 13.867: : 100it [08:47,  6.60s/it]

In [ ]:
sol

In [ ]:
ax = plt.subplot(111)
ax.plot(sol.t, sol.y.T[:, 0])
ax2 = plt.twinx(ax)
ax2.plot(sol.t, sol.y.T[:, 1], color='r')

ax.set_ylabel('Biomass', color='b')
ax2.set_ylabel('Artemisinic Acid', color='r')
